## Hyperparameter Tunning...
### How to find Hyperparameter?

In [22]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [7]:
model = Ridge()
model.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.0001}

In [8]:
df = pd.read_csv("datasets/advertising4D.csv")
df

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [9]:
X = df.drop("sales", axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [11]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Training with default Hyperparametrs.

In [13]:
model.fit(X_train, y_train)
r2 = model.score(X_test, y_test)
print(" R2 Score : ", r2)

 R2 Score :  0.8955000647586024


### Tunning Parameters.
#### i- alpha =1   and solver = 'auto'  ----> default

In [14]:
model = Ridge(alpha=1.0, solver='auto')
model.fit(X_train, y_train)
r2 = model.score(X_test, y_test)
print(" R2 Score : ", r2)

 R2 Score :  0.8955000647586024


#### ii- alpha =1000  and solver = 'auto'

In [15]:
model = Ridge(alpha=1000, solver='auto')
model.fit(X_train, y_train)
r2 = model.score(X_test, y_test)
print(" R2 Score : ", r2)

 R2 Score :  0.21986893721417622


#### iii- alpha =100  and solver = 'lsqr'

In [16]:
model = Ridge(alpha=100, solver='lsqr')
model.fit(X_train, y_train)
r2 = model.score(X_test, y_test)
print(" R2 Score : ", r2)

 R2 Score :  0.7741183584167997


### Cross Validate...

In [19]:
cv_score = cross_val_score(estimator=Ridge(alpha=100, solver='lsqr'), X=X_train, y=y_train, scoring='r2', cv=5)
print(cv_score)
print("R2 Score : ", np.mean(cv_score))

[0.79288744 0.69961578 0.72075948 0.69797299 0.69914695]
R2 Score :  0.7220765305189867


###  USE LOOPs...

In [20]:
b

1000 lsqr  :  0.2198689372141761
1000 svd  :  0.2198689372141761
100 lsqr  :  0.7741183584167997
100 svd  :  0.7741183584167998
10 lsqr  :  0.8935965814955683
10 svd  :  0.8935965814955684
5 lsqr  :  0.8952341896782466
5 svd  :  0.8952341896782466
1 lsqr  :  0.8955000647586024
1 svd  :  0.8955000647586024
0 lsqr  :  0.8954015893231161
0 svd  :  0.8954015893231162
0.8 lsqr  :  0.8954859417859493
0.8 svd  :  0.8954859417859494
0.5 lsqr  :  0.8954595678400928
0.5 svd  :  0.8954595678400928
0.2 lsqr  :  0.8954269006591332
0.2 svd  :  0.8954269006591332


In [21]:
alpha_list=[1000, 100, 10, 5, 1, 0, 0.8, 0.5, 0.2]
solver_list=['lsqr','svd']
for a in alpha_list:
    for s in solver_list:
        cv_score = cross_val_score(estimator=Ridge(alpha=a, solver=s), X=X_train, y=y_train, scoring='r2', cv=5)
        print(a,s,":", np.mean(cv_score))

1000 lsqr : 0.1567128382043864
1000 svd : 0.15671557983160506
100 lsqr : 0.7220765305189867
100 svd : 0.7220765305189862
10 lsqr : 0.8846156635849198
10 svd : 0.8846156635849198
5 lsqr : 0.8877716556780915
5 svd : 0.8877716556780915
1 lsqr : 0.8887489115631121
1 svd : 0.8887489115631123
0 lsqr : 0.8887400364939853
0 svd : 0.8887400364939854
0.8 lsqr : 0.8887558175618911
0.8 svd : 0.8887558175618914
0.5 lsqr : 0.8887581067042678
0.5 svd : 0.888758106704268
0.2 lsqr : 0.8887505806807606
0.2 svd : 0.8887505806807606


## GridSearch CV...

In [24]:
params = {"alpha":[1000, 100, 10, 1, 0.5],
         "solver": ['lsqr','svd']}
gs = GridSearchCV(estimator=Ridge(),
                 param_grid=params,
                 cv=5,
                 n_jobs=-1)#none means don't run the jobs on multiple cpu's
gs.fit(X_train, y_train)
print("Best Score: ", gs.best_score_)
print("Bes Params: ", gs.best_params_)

Best Score:  0.888758106704268
Bes Params:  {'alpha': 0.5, 'solver': 'svd'}


## RandomizedSearchCV...

In [26]:
params = {"alpha":[1000, 100, 10, 1, 0.5],
         "solver": ['lsqr','svd']}
rans = RandomizedSearchCV(estimator=Ridge(),
                 param_distributions=params,
                 n_iter=6, #no of models to try
                 scoring='r2',
                 cv=5,
                 n_jobs=-1)#none means don't run the jobs on multiple cpu's
rans.fit(X_train, y_train)
print("Best Score: ", rans.best_score_)
print("Bes Params: ", rans.best_params_)

Best Score:  0.888758106704268
Bes Params:  {'solver': 'svd', 'alpha': 0.5}
